## Submission

My final choice of strategy is : [27,31,38,1,1,1,1,0,0,0]

## Initial Thoughts

The first strategy that sprung to mind was concentrating resources in the higher value castles, with some in a couple of lower castles spaced so as to interrupt any streak of 3 the opponent might have. ie many of the lower castles have 0 soldiers allocated, with soldiers only in the 3rd and then 6th - 10th castles.



Then I realised that against a 'random' opponent (number of soldiers at each castle is randomly distributed in some way with mean 10), that the best strategy is to concentrate all soldiers in the first three castles which will win the game with maximum score on most occasions.

I think perhaps that all highest performing strategies are aimed at trying to win 3 consecutive castles as soon as possible, along with frustrating the opponent's attempt at this strategy.

The basic form of this strategy (ie concentrating all soldiers in the first 3 castles) I imagine will have been recognised by all of the Jane Streeters who will serve as opponents, so this is unlikely to work well due to counterplay.

I would identify the 3rd castle as being a particularly hard fought battleground due to its importance as being the highest value castle in the first 3, so if one wants to frustrate the opponent's 1,2,3 streak then that is the last opportunity, and being the highest valued offers highest returns of the 3 for winning.

This means on balance it may be too expensive in terms of soldiers to be worth contesting, ie resources can be focussed where the opponents soldiers are weaker to achieve better results

Therefore my initial guess at an entry is something like 35 soldiers in the 2nd castle, then 25 in the 4th castle and 20 each in the 5th and 6th. i.e [0,35,0,25,20,20,0,0,0,0]

Also it's occurred to me that perhaps people are much more likely to choose round numbers, meaning something like 46 has the same power level as 50 and is not much stronger than 41, so it may be worth optimising around this.

## Approach

While the strategy initially thought of seems ok, I thought it best to write some code to come up with a more empirical strategy. In the end I created an evolutionary algorithm that starts with some random strategies, selects the best performing ones in each generation and fills up the next generation with mutated versions of these (as well as the unaltered best strategies themselves).

I tried quite a few different variants, some of which came up with something along the lines of my initial strategy, however in the end I thought the best way of doing it was to create a set of strategies that I thought might look like the set of Jane Streeters' strategies, and score the generations against that. This gave the strategy [27,31,38,1,1,1,1,0,0,0], which I have submitted unaltered. On the face of it, it seems like it would be worse than my initial guess, as it loses one-on-one. However, when it comes to average score against a large number of varied strategies, it is vastly superior, and ultimately this is the goal of the submission which perhaps I failed to fully appreciate at first.

Everything I tried is included below with explanations.

## Simulation

I considered trying to use higher level libraries (I discovered there is one specifically for evolutionary algorithms), but ultimately I decided that what I had in mind was quite intuitive and relatively simple, and just using numpy would allow me to tweak every aspect of it (and be a more fun challenge). On the flip side, it may be quite far from the optimum in terms of time and memory complexity that might have been achieved with a well built library.

In [13]:
#importing potentially useful libraries
import numpy as np
import random

In [14]:
#function for calculating the score of a player against a particular opponent
def score(player,opponent):
    pstreak = 0
    ostreak = 0
    pscore = 0
    for i in range(0,10):
        if player[i] > opponent[i]:
            pscore += (i + 1)
            pstreak += 1
            ostreak = 0
        elif opponent[i] > player[i]:
            pstreak = 0
            ostreak += 1
        else:
            pstreak = 0
            ostreak = 0
            
        if ostreak == 3:
            return pscore
        if pstreak == 3:
            return pscore + sum(range(i+2,11))
    
    return pscore

In [12]:
#function for determining the winner of two strategies. Returns 1 if first argument wins, 0 if it doesn't (loses or ties).
#Not actually used in algorithm; just for testing.
def winner(player1,player2):
    score1 = score(player1,player2)
    score2 = score(player2,player1)
    if score1 > score2:
        return 1
    else:
        return 0

In [11]:
#function that returns the number of wins of a particular strategy 'strat' against a generation of strategies
def numwins(strat,generation):
    wins = 0
    for i in range(0,generation.shape[0]):
        wins += winner(strat,generation[i,:])
    
    return wins


In [10]:
#creates and fills an array of size 10 that sums to 100, using random.choice for randomness, returns this array
def randomfill():
    arr = np.zeros(10,dtype = np.uint8)
    indexes = [0,1,2,3,4,5,6,7,8,9]
    for i in range(0,100):
        arr[random.choice(indexes)] += 1
        
    return arr

In [9]:
#creates a generation of size 'gensize' of random strategies
def random_generation(gensize):
    gen = np.empty((gensize,10),dtype = np.uint8)
    for i in range(0,gensize):
        gen[i] = randomfill()
    
    return gen

In [8]:
#function that calculates the total score of a strategy against a generation
def generation_score(strat,generation):
    total = 0
    for gen in generation:
        total += score(strat,gen)
        
    return total   

In [59]:
#function that chooses the generators for the next generation, taking the generation, number of desired generators 'ngen'
#and generation size 'gensize'. Note: does not output generators in any order of scores
def next_generators(generation,ngen,gensize):
    scores = np.empty(gensize,dtype = np.uint64)
    for i in range(0,gensize):
        scores[i] = generation_score(generation[i],generation)
    
    indexes = (-scores).argsort()[:ngen]
    return generation[indexes].copy()        

In [6]:
#function that takes a strategy and noise amount and returns a mutated version of the strategy with noise applied
def mutate(strat,noise):
    out = strat.copy()
    indexes = [0,1,2,3,4,5,6,7,8,9]
    for i in range(0,noise):
    
        completed = False
        
        while not completed:
            idx = random.choice(indexes)
            if out[idx] > 0:
                out[idx] -=1
                completed = True
                
        completed = False
        
        while not completed:
            idx = random.choice(indexes)
            if out[idx] < 100:
                out[idx] +=1
                completed = True      
        
        
    return out

In [10]:
#function that generates the next generation given a set of generators
def nextgen_from_generators(generators,ngen,gensize,noise):
    nextgen = np.empty((gensize,10),dtype = np.uint8)
    factor = gensize // ngen
    for i in range(0,ngen):
            for j in range(0,factor):
                nextgen[i*factor + j] = mutate(generators[i],noise)
    
    return nextgen.copy()

In [5]:
#all in one generation function that generates the next generation given the current one; to use for debugging
def generate(generation,ngen,gensize,noise,verbose):
    generators = next_generators(generation,ngen,gensize).copy()
    if verbose:
        print(generators[0])
        
    return nextgen_from_generators(generators,ngen,gensize,noise)

In [13]:
#will train model for a specified number of generations, and output the highest performing strategy of the last model
def train(generationsize,ngenerators,noise_size,number_of_generations,verbose = True):
    #generating some random strategies
    gen = random_generation(generationsize)
    
    
    for i in range(0,number_of_generations):
        generators = next_generators(generation,ngen,gensize).copy()
        if verbose:
            print(generators[0])
            
        gen = nextgen_from_generators(generators,ngen,gensize,noise).copy()
        
    return next_generators(gen,16,generationsize)[0]

In [32]:
train(256,16,False,20,[0,35,0,25,20,20,0,0,0,0],200)

[15 14 14 14  8  6 10  4  8  7]
[ 4  6 17 17 13 13  2 10 10  8]
[ 5  8 20 18 14 13  6  7  7  2]
[ 4 11 24 20 14 11  3  6  6  1]
[ 6  6 26 20 17 12  5  1  7  0]
[ 3  7 29 24 16  9  1  6  2  3]
[10 12 32 16 16  9  1  2  1  1]
[ 9 18 35 15  5  0  8  5  4  1]
[12 19 39 15  2  0  4  5  4  0]
[ 1 19 41 20  9  0  1  2  0  7]
[10 21 45 15  1  3  2  0  1  2]
[ 4  8 48 19 11  3  4  0  2  1]
[ 2 14 49 22 12  0  1  0  0  0]
[12 22 52  4  1  2  5  1  1  0]
[12 23 54  4  1  0  5  1  0  0]
[13 20 56  1  2  0  7  0  1  0]
[ 2  0 60 21 13  0  0  2  0  2]
[ 9 13 63  7  3  0  2  0  2  1]
[10 14 64  3  0  4  0  3  1  1]
[ 0  3 67 16  9  3  0  1  0  1]
[ 0  4 70 10  8  2  2  2  1  1]
[ 7 18 72  1  0  1  0  0  0  1]
[ 5 14 75  3  0  3  0  0  0  0]
[ 2  6 76 10  4  0  0  1  0  1]
[ 4 10 78  0  1  1  4  1  1  0]
[ 4  4 80  4  4  1  1  1  1  0]
[ 0  3 80  7  4  2  1  0  2  1]
[ 5  7 83  1  2  1  0  0  1  0]
[ 3  6 85  2  0  0  2  1  1  0]
[ 1  0 85  2  3  3  2  1  1  2]
[ 0  1 87  5  3  1  2  0  1  0]
[ 2  5 8

array([[ 3,  4, 78,  6,  6,  3,  0,  0,  0,  0],
       [ 2,  3, 75,  5,  5,  4,  3,  0,  1,  2],
       [ 4,  0, 77,  6,  4,  3,  2,  2,  0,  2],
       [ 2,  5, 76,  4,  6,  3,  0,  3,  0,  1],
       [ 5,  2, 72,  8,  2,  5,  1,  3,  2,  0],
       [ 2,  5, 73,  7,  1,  4,  3,  2,  0,  3],
       [ 2,  5, 81,  4,  3,  2,  1,  1,  0,  1],
       [ 2,  4, 77,  4,  3,  5,  3,  1,  0,  1],
       [ 3,  3, 79,  7,  3,  2,  1,  0,  1,  1],
       [ 1,  4, 78,  5,  2,  4,  2,  1,  1,  2],
       [ 1,  4, 80,  5,  3,  1,  1,  2,  1,  2],
       [ 3,  3, 78,  4,  4,  2,  2,  1,  0,  3],
       [ 3,  0, 80,  5,  4,  5,  1,  1,  1,  0],
       [ 3,  0, 79,  7,  6,  1,  2,  0,  1,  1],
       [ 3,  0, 79,  4,  6,  2,  1,  1,  2,  2],
       [ 2,  3, 80,  4,  5,  6,  0,  0,  0,  0]], dtype=uint8)

In this form the model doesn't converge, as it only has its own generations as a reference so is constantly developing counterplay against itself.
As intuition would suggest, the 3rd castle seems to be an important battleground. The strategies generated here seem to oscillate between putting almost all soldiers into the 3rd castle, and then counterstrategies to that, and then various counterstrategies to counterstragies, before returning to the 3rd castle.

One possible way of making the model converge is by adding a significant chunk of random strategies to every generation.

Also to aid stability I think it's probably best to include the generating strategies 'as is' without any noise. Before I was just using noisy versions of the generating strategies without the actual strategies themselves.

In [37]:
#function that generates the next generation given a set of generators, same as the one previously defined but takes argument
#'inject_random' which if true means the next generation consists of half random strategies
def nextgen_from_generators(generators,ngen,gensize,noise,inject_random = True):
    nextgen = np.empty((gensize,10),dtype = np.uint8)
    if inject_random:
        factor = gensize // (2*ngen)
        for i in range(0,ngen):
            nextgen[i*factor] = generators[i]
            for j in range(1,factor):
                nextgen[i*factor + j] = mutate(generators[i],noise)
                
        nextgen[(gensize//2):] = random_generation(gensize//2)
            
    else:
        factor = gensize // ngen
        for i in range(0,ngen):
            nextgen[i*factor] = generators[i]
            for j in range(1,factor):
                nextgen[i*factor + j] = mutate(generators[i],noise)
    
    return nextgen.copy()

In [38]:
train(512,16,20,200)

[ 7 15 15 15 11  5  6  7  7 12]
[ 9 17 17 16  4  6  4 12  6  9]
[15 21 20  8  9 13  3  3  0  8]
[ 7 17 23 16 12 13  5  5  1  1]
[ 0 15 25 24 20  2  0  5  6  3]
[18 20 28  6 11 10  0  3  2  2]
[10 20 32 18 11  3  3  0  1  2]
[ 1  2 34 23 21  1  1  7  7  3]
[ 5 23 36 20 14  0  0  1  0  1]
[ 5 22 38 19 11  2  1  0  1  1]
[19 25 40  0  5  0 10  0  0  1]
[ 0  5 44 24 20  2  2  1  0  2]
[17 25 46  3  6  3  0  0  0  0]
[17 19 50  3  5  1  1  1  2  1]
[ 2  1 53 22 18  1  0  0  1  2]
[19 25 51  3  0  1  0  1  0  0]
[17 22 55  2  4  0  0  0  0  0]
[ 1  4 57 19 16  1  0  2  0  0]
[18 21 59  0  1  0  0  0  0  1]
[17 19 50  6  4  2  0  0  0  2]
[14 19 49  4  6  4  2  1  0  1]
[19 21 56  0  0  2  1  1  0  0]
[19 19 43  3  5  5  4  0  1  1]
[19 18 40  6  8  4  3  1  1  0]
[20 22 34  4  4  6  4  3  1  2]
[23 15 41  9  7  3  1  0  1  0]
[24 27 45  0  3  0  1  0  0  0]
[19 22 47  3  3  1  1  1  3  0]
[22 25 34  4  4  4  4  0  0  3]
[19 23 39  0  8  6  4  0  0  1]
[16 20 40  0  9  5  5  0  2  3]
[21 17 4

array([ 1, 23, 44, 28,  1,  0,  1,  1,  1,  0], dtype=uint8)

In [40]:
train(256,16,20,256)

[10 13 17 14  8  7  9 11  9  2]
[11 15 18 14  6  4 10 11 11  0]
[ 8 15 19 15  8  5  9 12  6  3]
[14 18 20 14  6  4  7  9  6  2]
[16 18 23 11  4  4  5  6  6  7]
[16 21 23 14  8  3  8  2  4  1]
[ 3  3 29 18 15 12  3 12  0  5]
[ 1  2 32 19 16  9  4  9  3  5]
[ 2  1 34 21 19  7  5  5  1  5]
[19 19 35  4  1  6  3  4  7  2]
[16 20 37  7  1  7  1  4  7  0]
[ 1  2 42 20 17  2  1  5  5  5]
[ 2  4 43 20 17  0  4  5  3  2]
[21 20 49  4  0  0  2  1  2  1]
[20 20 50  3  2  1  3  1  0  0]
[ 0 14 53 17  9  0  0  2  0  5]
[ 1 15 54 17  7  1  0  2  2  1]
[14 18 58  0  0  2  4  3  0  1]
[ 1 20 58 18  1  1  0  0  1  0]
[ 0  5 59 19 14  0  0  2  0  1]
[ 1  0 57 20 14  5  2  0  0  1]
[ 2  0 58 20 16  1  1  0  0  2]
[ 3  0 53 22 15  1  1  1  4  0]
[ 2  3 57 22 14  0  0  1  0  1]
[ 0  4 50 20 17  5  3  1  0  0]
[ 3  3 46 18 18  6  4  2  0  0]
[ 1  5 44 22 20  7  1  0  0  0]
[ 5  6 55 17 15  0  0  0  2  0]
[ 7 11 56 16  9  1  0  0  0  0]
[ 3 11 13  9 22 18 13 10  0  1]
[ 1 13 14 11 24 17 12  3  2  3]
[ 3 16 3

array([18,  7, 13, 25, 19, 11,  0,  1,  5,  1], dtype=uint8)

In [41]:
train(256,16,20,512)

[11 16 17 12  8 11  5  8  2 10]
[ 6 17 16 18  5 14  4  6  8  6]
[ 8 22 19 15  7  8  4  4  2 11]
[11 22 21 12  5  7  3  3  1 15]
[12 21 23 12 13  4  1  9  5  0]
[13 23 26 12 13  3  1  3  4  2]
[11 22 29 15 10  6  2  2  1  2]
[15 26 30 13  5  8  1  1  1  0]
[ 2 23 33 21  1 14  3  0  3  0]
[11 24 35 15  2  8  1  0  3  1]
[16 27 36 14  0  4  0  1  0  2]
[15 26 38 15  3  0  0  1  0  2]
[ 1 24 40 21  4  6  2  1  0  1]
[ 2 23 43 20  3  2  0  0  5  2]
[ 2 16 45 21  9  2  0  4  0  1]
[ 3 21 46 21  0  5  3  1  0  0]
[ 3 22 48 21  1  3  0  0  2  0]
[ 1 19 52 20  3  3  1  0  0  1]
[ 5 21 53 17  3  0  0  1  0  0]
[ 5 21 53 17  3  0  0  1  0  0]
[ 1 19 41 24  5  5  0  0  4  1]
[ 0 21 41 21  4  4  4  1  2  2]
[ 3 22 43 19  2  2  5  4  0  0]
[ 0 18 42 27  8  3  1  1  0  0]
[ 5 21 45 20  2  2  4  1  0  0]
[ 3 24 48 20  0  2  2  0  0  1]
[ 4 25 49 16  1  2  0  0  3  0]
[ 4 29 49 14  0  0  1  1  2  0]
[ 2 29 24 20 10  3  2  3  7  0]
[ 0 29 25 20  9  8  5  3  1  0]
[ 3 30 32 21  6  0  3  4  1  0]
[ 5 32 3

[ 6 29 38 19  2  3  1  1  0  1]
[ 7 28 40 17  6  0  0  1  0  1]
[ 1 28 30 18 10  6  4  1  0  2]
[ 0 32 16 16 13  8  2  8  2  3]
[10 20 32 19 14  0  1  0  3  1]
[ 5 13 35 30 14  1  1  1  0  0]
[ 8 19 37 17  5  5  4  3  0  2]
[ 3 25 21 18 14  8  7  4  0  0]
[ 1 29 21 19 16  7  3  2  2  0]
[ 2 31 23 21 17  6  0  0  0  0]
[ 0 19 27 22 18  3  3  6  2  0]
[10 28 30 16 14  1  0  1  0  0]
[13 28 31 12 10  3  0  1  1  1]
[12 23 33 13 12  2  2  0  1  2]
[ 3 13 36 25 15  1  1  0  3  3]
[ 3 16 40 23 14  1  0  2  0  1]
[11 20 41 15  6  1  3  1  2  0]
[10 19 43 17  6  1  3  0  1  0]
[ 1 11 44 23 17  1  1  1  1  0]
[16 28 46  4  1  1  1  0  0  3]
[15 29 50  1  2  1  0  0  0  2]
[15 28 51  0  2  0  0  0  2  2]
[13 27 56  0  0  1  0  0  2  1]
[13 26 61  0  0  0  0  0  0  0]
[14 21 58  0  0  2  2  0  2  1]
[15 19 58  1  1  3  1  1  1  0]
[ 0  3 62 16 15  1  1  0  1  1]
[ 0  1 60 20 17  1  0  1  0  0]
[ 2  1 57 19 19  2  0  0  0  0]
[ 5  9 15 12 22  3 11 12  5  6]
[ 4  9 13 16 21  9  5  7  3 13]
[ 6  6 1

array([ 9, 21, 41, 19,  1,  0,  3,  1,  4,  1], dtype=uint8)

This works much better and the model seems to converge to something like 20 40 20 on the 2nd 3rd and 4th castle. However there are notable 'crises of confidence' along the way. Something that might fix this is feeding in a history of best performing strategies of every previous generation, or just including some of the generated strategies by the above method with every generation, in the same spirit as including random strategies with every generation

I have to revisit two functions in order to implement this:

In the nextgen_from_generators function I've added 'past' to the arguments, and the function now takes this history and concatenates it to the end of the output 'nextgen'

In [58]:
def nextgen_from_generators(past,generators,ngen,gensize,noise,inject_random = True):
    nextgen = np.empty((gensize + (past.shape)[0],10),dtype = np.uint8)
    nextgen[gensize:] = past.copy()
    if inject_random:
        factor = gensize // (2*ngen)
        for i in range(0,ngen):
            nextgen[i*factor] = generators[i]
            for j in range(1,factor):
                nextgen[i*factor + j] = mutate(generators[i],noise)
                
        nextgen[(gensize//2):gensize] = random_generation(gensize//2)
            
    else:
        factor = gensize // ngen
        for i in range(0,ngen):
            nextgen[i*factor] = generators[i]
            for j in range(1,factor):
                nextgen[i*factor + j] = mutate(generators[i],noise)
    
    return nextgen.copy()

In the 'train' function I've added a few lines to initialise the history array and copy and add to it in the main loop.
Another key change is calling the 'next_generators' function with gensize + i. This avoids having to change anything in the 'next_generators' function.

In [13]:
#will train model for a specified number of generations, and output the highest performing strategy of the last model
def train(gensize,ngenerators,noise,number_of_generations,verbose = True):
    #generating some random strategies
    gen = random_generation(gensize)
    
    generators = next_generators(gen,ngenerators,gensize)
    if verbose:
        print(generators[0])
    
    past = np.array([generators[0]])
    gen = nextgen_from_generators(past,generators,ngenerators,gensize,noise)
    
    for i in range(1,number_of_generations):
        
        generators = next_generators(gen,ngenerators,gensize + i)
        if verbose:
            print(generators[0])
            
        temphist = past.copy()
        past = np.append(temphist, [generators[0].copy()], axis = 0)
            
        gen = nextgen_from_generators(past,generators,ngenerators,gensize,noise)
        
    return next_generators(gen,16,gensize)[0]
    

In [42]:
train(256,16,20,256)

[ 7 15 13 14 10 13  4  8 12  4]
[ 6 12 21 17 12  7  9  6  6  4]
[ 5  9 23 20 15  6  6  5  7  4]
[ 6  9 24 21 17  5  6  5  5  2]
[ 2 10 25 25 19  1  5  6  5  2]
[ 2  4 28 24 20  2  5  4  8  3]
[10 13 30 18 14  0  5  2  6  2]
[ 8 14 35 16 13  1  4  1  8  0]
[ 7 17 34 16 13  2  3  2  6  0]
[ 7  4 37 25 21  4  0  0  1  1]
[ 1 19 41 24  2  3  0  2  3  5]
[ 0 21 41 27  1  2  0  1  2  5]
[ 1 20 44 20  1  5  4  3  2  0]
[ 0 18 48 30  1  0  1  1  1  0]
[ 5 21 47 18  1  2  3  1  2  0]
[ 4 25 50 18  0  0  0  0  3  0]
[ 4 25 50 18  0  0  0  0  3  0]
[ 2 18 54 21  2  1  0  1  0  1]
[ 7  1 31 28 24  5  1  0  0  3]
[ 9  5 38 24 20  3  0  0  0  1]
[ 1 16 44 30  4  1  2  0  1  1]
[ 4 25 50 18  0  0  0  0  3  0]
[ 4 25 50 18  0  0  0  0  3  0]
[ 7  1 31 28 24  5  1  0  0  3]
[ 9  5 38 24 20  3  0  0  0  1]
[ 0 18 48 30  1  0  1  1  1  0]
[ 0 20 48 30  0  0  0  2  0  0]
[ 4 25 50 18  0  0  0  0  3  0]
[ 7  1 31 28 24  5  1  0  0  3]
[ 9  5 38 24 20  3  0  0  0  1]
[ 0 20 48 30  0  0  0  2  0  0]
[ 0 17 4

array([19, 27, 32,  1,  8,  5,  4,  3,  0,  1], dtype=uint8)

This produces an interesting result. The model seems to get to a point where it is alternating between roughly three distinct strategies: [15,17,53,5,5,2,2,0,0,1], [5,5,30,25,22,9,2,2,0,0] and [0,0,57,14,15,7,3,2,1,1], with some variation in the exact numbers, but with all strategies following approximately these shapes.

I think it's best to run the model for a bit longer to see if this behaviour continues.

In [43]:
train(256,16,20,512)

[ 7  5 14 14 14  9 13 11  6  7]
[11 15 16 15  5  5  6  9 14  4]
[11 15 17 16 17  3 13  5  1  2]
[14 19 21  6  3 10  7  2 10  8]
[15 21 21  5  4  9  6  2  9  8]
[10 17 22 17  8 10  9  4  0  3]
[ 8 17 24 18  7 11 11  3  0  1]
[ 9 12 28 18 16  3  3  3  4  4]
[13 14 29 18 14  2  0  1  5  4]
[18 22 31  7  6  9  2  2  3  0]
[17 17 32  9  2 12  5  4  1  1]
[18 23 35  6  0 12  0  2  1  3]
[18 23 38  1  0 11  0  4  1  4]
[18 20 40  2  1 10  1  4  1  3]
[17 20 43  3  2 10  3  1  0  1]
[16 19 46  1  2 10  0  1  3  2]
[17 15 47  4  4  7  1  2  2  1]
[18 21 49  2  2  4  0  0  4  0]
[18 19 52  1  3  6  0  0  1  0]
[18 22 31  7  6  9  2  2  3  0]
[ 1  1 54 19 20  1  1  1  1  1]
[ 0  4 56 20 16  0  0  2  0  2]
[ 3 12 28 20 19  1  3  6  5  3]
[18 19 52  1  3  6  0  0  1  0]
[17 20 53  3  3  1  0  1  1  1]
[18 22 31  7  6  9  2  2  3  0]
[ 0  4 56 20 16  0  0  2  0  2]
[ 3 12 28 20 19  1  3  6  5  3]
[ 1 13 26 20 19  2  5  8  5  1]
[17 20 53  3  3  1  0  1  1  1]
[ 1  1 54 19 20  1  1  1  1  1]
[ 0  1 5

[ 2 19 45 26  5  0  1  1  0  1]
[ 1 29 25 27  5  5  5  3  0  0]
[ 5 32 36 21  0  1  2  1  1  1]
[ 1 16 42 28  8  0  3  1  0  1]
[ 4 21 46 23  5  0  1  0  0  0]
[ 2 22 28 29  7  3  3  4  2  0]
[ 9 34 37 16  1  0  1  0  1  1]
[ 1 16 42 28  8  0  3  1  0  1]
[ 4 21 46 23  5  0  1  0  0  0]
[ 5 31 33 22  1  3  2  1  2  0]
[ 9 34 37 16  1  0  1  0  1  1]
[ 1 16 42 28  8  0  3  1  0  1]
[ 4 21 46 23  5  0  1  0  0  0]
[ 0 24 27 31  8  4  3  3  0  0]
[ 5 31 33 22  1  3  2  1  2  0]
[ 1 16 42 28  8  0  3  1  0  1]
[ 4 21 46 23  5  0  1  0  0  0]
[ 0 24 27 29  7  2  5  4  2  0]
[ 9 34 37 16  1  0  1  0  1  1]
[ 2 19 45 26  5  0  1  1  0  1]
[ 4 21 46 23  5  0  1  0  0  0]
[ 0 23 26 29  9  6  3  1  2  1]
[ 5 31 33 22  1  3  2  1  2  0]
[ 2 19 45 26  5  0  1  1  0  1]
[ 4 21 46 23  5  0  1  0  0  0]
[ 0 24 27 31  8  4  3  3  0  0]
[ 9 34 37 16  1  0  1  0  1  1]
[ 2 19 45 26  5  0  1  1  0  1]
[ 4 21 46 23  5  0  1  0  0  0]
[ 0 24 27 31  8  4  3  3  0  0]
[ 9 34 37 16  1  0  1  0  1  1]
[ 4 21 4

array([20,  3,  5, 37, 18, 15,  2,  0,  0,  0], dtype=uint8)

Soon after the 256th generation the model seems to have found roughly three strategies focussed on the 2nd,3rd and 4th castles. The reoccurring ones are roughly [1,16,47,28,7,0,0,0,1,0], [9,34,37,16,1,0,1,0,1,], and [1,23,17,34,9,5,4,5,2,0]. However, very near the end of this run it finds an entirely new strategy of ignoring the 2nd and 3rd castles almost entirely, contesting only the first one to disrupt the 3 castle streak at the start, and then putting the rest in the 4th,5th and 6th to win that streak, which is remarkably (and satisfyingly) similar to the strategy I first came up with after some initial thought.

It seems this model only finds new strategies every few hundred generations, which makes me think I should try running it for much longer.

A potential issue is that as the algorithm goes on longer and longer, the more the history dominates the set of strategies in each generation, making the random strategies and generated strategies less important to the total score. I would think there is not a problem with the generated strategies being less of a proportion, as their main purpose is not to be trained against. There may be an issue with the random strategies losing their clout, as it may lead to strategies being generated that lose to naive strategies. 

I will try continuing with the algorithm unchanged

In [14]:
train(256,16,20,1024)

[ 7 14 16 14  7 11  4  7 10 10]
[ 8 14 16 16  9 13  4  4  8  8]
[ 9 17 17 18  5  3  4  9 10  8]
[ 1 10 21 17 17  0 12  9  9  4]
[16 18 21  9  9  7 10  0  7  3]
[17 19 25  4  9  7  4  2  8  5]
[ 3 18 27 21 11  3  0  0  7 10]
[ 9 13 31 20 14  0  0  1  7  5]
[ 4 11 32 22 15  3  1  2  6  4]
[17 23 34  3 11  6  0  2  2  2]
[19 23 37  5 10  4  0  2  0  0]
[18 25 39  6  8  1  2  1  0  0]
[ 3  5 43 23 19  2  1  1  1  2]
[ 0  6 44 24 20  1  0  3  0  2]
[16 20 47  3  9  1  3  0  0  1]
[ 2  5 48 24 18  1  0  1  1  0]
[ 3  7 49 21 18  0  1  0  0  1]
[16 13 54  1  9  1  2  2  1  1]
[ 5  3 54 21 15  2  0  0  0  0]
[17 14 57  2  4  2  1  3  0  0]
[14 15 58  3  2  1  4  3  0  0]
[ 0 20 25 20 11  7  3  1  5  8]
[ 2 19 25 23  9  7  3  3  2  7]
[18 25 39  6  8  1  2  1  0  0]
[ 3  7 49 21 18  0  1  0  0  1]
[14 15 58  3  2  1  4  3  0  0]
[16 17 57  2  0  0  3  3  0  2]
[ 0 19 24 24  8  7  5  2  3  8]
[18 25 39  6  8  1  2  1  0  0]
[ 2  5 48 24 18  1  0  1  1  0]
[ 2  8 42 22 19  4  2  0  0  1]
[16 17 5

[14 15 58  3  2  1  4  3  0  0]
[14 31 32 11  5  4  1  1  0  1]
[ 1  0 47 27 18  3  2  1  0  1]
[ 1  0 47 27 18  3  2  1  0  1]
[14 15 58  3  2  1  4  3  0  0]
[ 5 31 37 16  7  2  1  0  0  1]
[ 1  0 47 27 18  3  2  1  0  1]
[ 2  1 48 27 21  1  0  0  0  0]
[16 13 54  1  9  1  2  2  1  1]
[ 3 23 43 17  9  2  1  2  0  0]
[ 1  0 47 27 18  3  2  1  0  1]
[14 15 58  3  2  1  4  3  0  0]
[ 6 30 36 15  7  3  0  2  0  1]
[ 2  1 48 27 21  1  0  0  0  0]
[ 2  1 48 27 21  1  0  0  0  0]
[14 15 58  3  2  1  4  3  0  0]
[ 6 30 36 15  7  3  0  2  0  1]
[ 1  0 47 27 18  3  2  1  0  1]
[14 15 58  3  2  1  4  3  0  0]
[ 6 30 36 15  7  3  0  2  0  1]
[ 2  1 48 27 21  1  0  0  0  0]
[ 5  3 54 21 15  2  0  0  0  0]
[14 15 58  3  2  1  4  3  0  0]
[ 6 30 36 15  7  3  0  2  0  1]
[ 2  1 48 27 21  1  0  0  0  0]
[ 0  2 50 28 16  1  1  1  1  0]
[14 15 58  3  2  1  4  3  0  0]
[ 6 30 36 15  7  3  0  2  0  1]
[ 0  2 50 28 16  1  1  1  1  0]
[ 1  2 49 28 16  0  1  2  0  1]
[14 12 56  1  2  5  3  4  2  1]
[ 5 31 3

[ 6 20 15 33  4  8  7  4  2  1]
[ 0 32 18 33  3  3  5  4  2  0]
[ 3 34 21 28  0  3  6  3  1  1]
[ 6 20 15 33  4  8  7  4  2  1]
[ 6 22 16 30  4 11  5  5  1  0]
[ 6 21 17 34  4  8  6  3  1  0]
[ 3 34 21 28  0  3  6  3  1  1]
[ 6 20 15 33  4  8  7  4  2  1]
[ 0 32 18 33  3  3  5  4  2  0]
[ 3 34 21 28  0  3  6  3  1  1]
[ 6 20 15 33  4  8  7  4  2  1]
[ 8 21 19 31  5  7  4  3  2  0]
[ 3 34 21 28  0  3  6  3  1  1]
[ 6 20 15 33  4  8  7  4  2  1]
[ 6 22 17 35  5  4  5  4  1  1]
[ 1 32 14 29  5  5  5  5  2  2]
[ 6 21 18 30  5  8  7  5  0  0]
[ 3 34 21 28  0  3  6  3  1  1]
[ 6 21 18 30  5  8  7  5  0  0]
[ 6 18 17 34  6 10  4  3  1  1]
[ 7 22 21 30  1  8  4  4  2  1]
[ 0 33 20 31  1  6  5  4  0  0]
[ 6 18 17 34  6 10  4  3  1  1]
[ 6 22 17 35  5  4  5  4  1  1]
[ 7 21 19 29  3  9  6  6  0  0]
[ 7 23 12 33  5 11  5  4  0  0]
[ 3 34 21 28  0  3  6  3  1  1]
[ 6 18 17 34  6 10  4  3  1  1]
[ 7 22 21 30  1  8  4  4  2  1]
[ 7 21 19 29  3  9  6  6  0  0]
[ 7 23 12 33  5 11  5  4  0  0]
[ 3 34 2

[ 2 39  7  1 23 14 14  0  0  0]
[ 0 40  1  2 25 17 15  0  0  0]
[ 0 34  2  4 27 17 16  0  0  0]
[ 3 41  9  1 21 13 11  1  0  0]
[ 0 40  1  2 25 17 15  0  0  0]
[ 1 35  3  4 26 14 14  0  1  2]
[ 2 39  7  1 23 14 14  0  0  0]
[ 0 40  1  2 25 17 15  0  0  0]
[ 0 34  2  4 27 17 16  0  0  0]
[ 2 39  7  1 23 14 14  0  0  0]
[ 0 40  1  2 25 17 15  0  0  0]
[ 0 34  2  4 27 17 16  0  0  0]
[ 2 39  7  1 23 14 14  0  0  0]
[ 1 40  0  2 24 17 16  0  0  0]
[ 0 34  2  4 27 17 16  0  0  0]
[ 3 41  9  1 21 13 11  1  0  0]
[ 1 40  0  2 24 17 16  0  0  0]
[ 0 34  2  4 27 17 16  0  0  0]
[ 3 41  9  1 21 13 11  1  0  0]
[ 0 40  1  2 25 17 15  0  0  0]
[ 1 35  3  4 26 14 14  0  1  2]
[ 3 41  9  1 21 13 11  1  0  0]
[ 0 40  1  2 25 17 15  0  0  0]
[ 0 34  2  4 27 17 16  0  0  0]
[ 3 41  9  1 21 13 11  1  0  0]
[ 1 41  3  3 24 15 10  1  1  1]
[ 1 35  3  4 26 14 14  0  1  2]
[ 3 41  9  1 21 13 11  1  0  0]
[ 0 40  1  2 25 17 15  0  0  0]
[ 3 41  9  1 21 13 11  1  0  0]
[ 3 41  9  1 21 13 11  1  0  0]
[ 3 36  

array([ 5, 33,  4,  7, 29, 14,  7,  1,  0,  0], dtype=uint8)

This has reached a similar strategy as the previous run, so not a lot more information has been gained. It's clear I should  run the algorithm for much longer, however I would guess in its current form it has quadratic time complexity with the entire history fed in every time. I think a reasonable way around this is to take a random sample of the history to use, up to a maximum size. Also I would like to run it overnight so I will implement an option for it to run indefinitely.

I will also add the generation number to the print line so I can know how many generations the algorithm runs for

In [3]:
def sample(history,samplesize):
    nrows = history.shape[0]
    if nrows > samplesize:
        return history[np.random.choice(nrows,samplesize,replace = False)]
    else:
        return history

In [14]:
#will train model for a specified number of generations, and output the highest performing strategy of the last model
def train(gensize,ngenerators,noise,number_of_generations,hist_sample_size,verbose = True):
    #generating some random strategies
    gen = random_generation(gensize)
    
    generators = next_generators(gen,ngenerators,gensize)
    if verbose:
        print('0: ' + str(generators[0]))
    
    past = np.array([generators[0]])
    gen = nextgen_from_generators(past,generators,ngenerators,gensize,noise)
    
    for i in range(1,number_of_generations):
        
        generators = next_generators(gen,ngenerators,gensize + np.minimum(i,hist_sample_size))
        if verbose:
            print(str(i) + ': ' + str(generators[0]))
            
        past = np.append(past, [generators[0].copy()], axis = 0)
            
        gen = nextgen_from_generators(sample(past,hist_sample_size),generators,ngenerators,gensize,noise)
        
    return next_generators(gen,16,gensize)[0]
    

In [23]:
#will train model until manual interrupt
def train_until_interrupt(gensize,ngenerators,noise,hist_sample_size,verbose = True):
    #generating some random strategies
    gen = random_generation(gensize)
    
    generators = next_generators(gen,ngenerators,gensize)
    if verbose:
        print('0: ' + str(generators[0]))
    
    past = np.array([generators[0]])
    gen = nextgen_from_generators(past,generators,ngenerators,gensize,noise)
    
    i = 0
    while True:
        i += 1
        generators = next_generators(gen,ngenerators,gensize + np.minimum(i,hist_sample_size))
        if verbose:
            print(str(i) + ': ' + str(generators[0]))
            
        
        past = np.vstack((past, [generators[0].copy()]))
            
        gen = nextgen_from_generators(sample(past,hist_sample_size),generators,ngenerators,gensize,noise)
        
    return 0
    

In [24]:
train_until_interrupt(256,16,20,512)

0: [ 5  5 16 18 16  5  9 12  5  9]
1: [ 3  3 19 18 17  8  8 11  5  8]
2: [10  9 19 17 17  9  0  5  5  9]
3: [ 4  8 20 21 19 11  8  6  3  0]
4: [ 4  6 24 20 20  7  7  2  5  5]
5: [ 7 17 24 21 11  9  2  0  2  7]
6: [ 8 16 26 23  7  7  2  0  3  8]
7: [10 21 28 17 11  6  0  0  4  3]
8: [11 22 30 17  8  6  1  0  5  0]
9: [12 22 32 15 11  4  0  1  2  1]
10: [11 21 34 18  9  3  1  0  1  2]
11: [ 9 22 36 15  9  0  3  1  3  2]
12: [ 4 17 37 22 10  3  3  3  0  1]
13: [ 1 22 40 24 10  0  2  0  0  1]
14: [ 5 20 41 19  2  6  3  3  0  1]
15: [ 2 18 43 25  7  2  2  0  0  1]
16: [ 1 22 22 23 15  8  4  1  1  3]
17: [ 3 25 44 25  1  0  0  0  0  2]
18: [ 4 27 41 22  1  0  0  2  1  2]
19: [ 6 24 16 29  9  5  4  2  2  3]
20: [ 8 31 33 19  3  1  2  3  0  0]
21: [ 8 32 31 19  5  0  3  0  1  1]
22: [ 1 22 40 24 10  0  2  0  0  1]
23: [ 1 22 41 25  7  1  1  1  0  1]
24: [ 6 24 16 29  9  5  4  2  2  3]
25: [ 6 24 16 29  9  5  4  2  2  3]
26: [ 8 31 33 19  3  1  2  3  0  0]
27: [ 9 31 33 19  1  1  2  2  0  2]
28

225: [17  3  5 38 22 15  0  0  0  0]
226: [17  3  5 38 22 15  0  0  0  0]
227: [17  3  5 38 22 15  0  0  0  0]
228: [17  3  5 38 22 15  0  0  0  0]
229: [ 0 17 35 41  3  1  1  0  2  0]
230: [16  2  2 42 22 16  0  0  0  0]
231: [18  3  4 36 20 15  3  0  1  0]
232: [17  3  5 38 22 15  0  0  0  0]
233: [17  1  2 40 22 15  0  1  1  1]
234: [ 0 17 35 41  3  1  1  0  2  0]
235: [16  0  1 43 22 16  1  1  0  0]
236: [18  3  4 36 20 15  3  0  1  0]
237: [18  1  6 39 23 13  0  0  0  0]
238: [ 0 17 35 41  3  1  1  0  2  0]
239: [19  0  0 43 20 16  0  1  0  1]
240: [ 2 14 29 44  1  1  2  2  4  1]
241: [17  3  5 38 22 15  0  0  0  0]
242: [ 3 13 29 44  2  3  2  3  0  1]
243: [19  0  0 43 20 16  0  1  0  1]
244: [ 1 17 26 46  2  3  3  2  0  0]
245: [ 1 17 23 45  2  5  3  2  1  1]
246: [ 0 28 17 17 18 10  8  1  1  0]
247: [16  0  1 43 22 16  1  1  0  0]
248: [ 1 17 26 46  2  3  3  2  0  0]
249: [ 3 16 30 43  1  3  2  1  0  1]
250: [ 0 28 17 17 18 10  8  1  1  0]
251: [17  1  2 40 22 15  0  1  1  1]
2

447: [ 0 10 37 19  7 15  6  4  2  0]
448: [ 0 10 37 19  7 15  6  4  2  0]
449: [ 3 30 27 10  4 17  5  3  1  0]
450: [ 0 11 36 17  6 17  9  3  1  0]
451: [ 1 22 18 19  7 22  7  4  0  0]
452: [ 3 30 27 10  4 17  5  3  1  0]
453: [ 0 10 37 19  7 15  6  4  2  0]
454: [ 0  8 37 21  9 17  5  2  1  0]
455: [ 2  5 35 21 12 16  5  1  2  1]
456: [ 3 30 27 10  4 17  5  3  1  0]
457: [ 2  5 35 21 12 16  5  1  2  1]
458: [ 0  4 38 20 13 16  3  1  3  2]
459: [ 0  1 36 21 10 18  6  3  3  2]
460: [ 2  1 36 20 14 17  6  1  1  2]
461: [ 0  0 36 21 16 16  5  3  0  3]
462: [ 0  1 37 19 17 18  5  1  0  2]
463: [ 1  1 37 21 16 17  2  3  1  1]
464: [ 1  1 37 21 16 17  2  3  1  1]
465: [ 0  2 37 21 16 18  5  1  0  0]
466: [ 0  3 38 22 14 16  2  2  2  1]
467: [ 0  1 38 18 17 18  5  2  0  1]
468: [ 0  0 39 11 17 20  8  3  1  1]
469: [ 0  0 39 11 17 20  8  3  1  1]
470: [ 0  0 40  9 18 18 11  2  1  1]
471: [ 2  1 42  5 18 19 11  2  0  0]
472: [ 2  3 42  2 16 18 13  2  0  2]
473: [ 0  3 43  0 17 20 12  2  1  2]
4

669: [ 6 27 53 12  1  0  0  0  0  1]
670: [ 9 27 53  8  1  1  1  0  0  0]
671: [ 3  2 54  3  2 18 10  7  0  1]
672: [ 2  0 41  2 15 22 14  4  0  0]
673: [ 6 27 53 12  1  0  0  0  0  1]
674: [11 29 55  3  0  0  0  0  0  2]
675: [11 29 55  3  0  0  0  0  0  2]
676: [11 29 55  3  0  0  0  0  0  2]
677: [14 27 55  0  1  1  1  1  0  0]
678: [18 29 48  0  1  0  0  1  2  1]
679: [17 28 53  0  0  0  1  1  0  0]
680: [20 26 49  0  1  1  2  0  1  0]
681: [18 21 50  2  4  2  3  0  0  0]
682: [20 28 48  1  0  0  0  2  1  0]
683: [20 29 50  0  0  0  0  0  0  1]
684: [20 29 50  0  0  0  0  0  0  1]
685: [20 29 50  0  0  0  0  0  0  1]
686: [20 29 50  0  0  0  0  0  0  1]
687: [21 29 50  0  0  0  0  0  0  0]
688: [21 29 50  0  0  0  0  0  0  0]
689: [21 29 50  0  0  0  0  0  0  0]
690: [21 29 50  0  0  0  0  0  0  0]
691: [21 29 50  0  0  0  0  0  0  0]
692: [18 21 50  2  4  2  3  0  0  0]
693: [21 29 50  0  0  0  0  0  0  0]
694: [21 29 50  0  0  0  0  0  0  0]
695: [17 30 51  1  0  0  0  0  0  1]
6

891: [20 21 44  2  3  5  3  1  0  1]
892: [21 22 45  2  2  3  3  1  0  1]
893: [21 21 44  4  4  4  2  0  0  0]
894: [21 21 44  4  4  4  2  0  0  0]
895: [20 21 47  3  4  3  1  0  1  0]
896: [21 21 44  4  4  4  2  0  0  0]
897: [22 26 48  1  1  1  0  0  1  0]
898: [21 22 47  3  3  2  2  0  0  0]
899: [22 26 48  1  1  1  0  0  1  0]
900: [21 21 44  4  4  4  2  0  0  0]
901: [23 24 48  1  1  1  1  1  0  0]
902: [23 24 48  1  1  1  1  1  0  0]
903: [23 24 48  1  1  1  1  1  0  0]
904: [21 14 53  1  4  4  2  0  1  0]
905: [21 21 44  4  4  4  2  0  0  0]
906: [21 21 44  4  4  4  2  0  0  0]
907: [21 21 44  4  4  4  2  0  0  0]
908: [22 26 48  1  1  1  0  0  1  0]
909: [21 14 53  1  4  4  2  0  1  0]
910: [21 21 40  4  3  6  2  2  1  0]
911: [23 24 48  1  1  1  1  1  0  0]
912: [23 24 48  1  1  1  1  1  0  0]
913: [23 24 48  1  1  1  1  1  0  0]
914: [22 26 48  1  1  1  0  0  1  0]
915: [21 14 53  1  4  4  2  0  1  0]
916: [21 21 44  4  4  4  2  0  0  0]
917: [21 20 38  5  5  9  0  1  0  1]
9

...

13553: [33 17 24  8  9  6  0  0  1  2]
13554: [ 7 37 32 14  1  4  2  3  0  0]
13555: [20  0  0 42 15 18  2  3  0  0]
13556: [32 17 23  7  8  4  1  4  2  2]
13557: [ 0  0 51 16 22  0  2  4  3  2]
13558: [ 2  2 62  1  4 16  9  4  0  0]
13559: [20  1  0 41 16 18  0  1  2  1]
13560: [32 17 21  8  8  5  2  6  0  1]
13561: [ 0  6 54 26 13  0  1  0  0  0]
13562: [19  0  2 41 21 14  2  1  0  0]
13563: [32 15 23  8 10  8  0  0  2  2]
13564: [ 0  0 51 16 22  0  2  4  3  2]
13565: [ 4  4 61  0  4 14  8  5  0  0]
13566: [ 0  6 39  0 18 17 14  4  1  1]
13567: [32 15 23  8 10  8  0  0  2  2]
13568: [ 8 40 30 14  2  4  0  0  1  1]
13569: [ 0  1 48  0 12 19 15  5  0  0]
13570: [22 17 56  0  0  2  1  2  0  0]
13571: [ 2  2 61  3  4 11  9  7  1  0]
13572: [20  0  0 42 15 18  2  3  0  0]
13573: [30  4  3 20 20 21  0  0  2  0]
13574: [33 17 25  7  8  5  0  0  3  2]
13575: [ 0  1 61  4  5 13 10  5  1  0]
13576: [ 0  4 41  1 17 20 13  2  0  2]
13577: [ 1  4 23 43 23  4  0  1  0  1]
13578: [ 0  6 54 26 13  0

13764: [ 0  0 51 16 22  0  2  4  3  2]
13765: [ 7  0 16 41 28  8  0  0  0  0]
13766: [32 17 23  7  9  8  1  0  3  0]
13767: [ 8 40 29 15  3  2  1  2  0  0]
13768: [ 0  2 54 17 21  0  1  2  2  1]
13769: [33 17 24  8  9  6  0  0  1  2]
13770: [ 8 39 28 15  1  5  1  2  1  0]
13771: [ 0  6 54 26 13  0  1  0  0  0]
13772: [ 5  9 59 15 12  0  0  0  0  0]
13773: [ 1 30 12 16 26  5  3  4  2  1]
13774: [ 8 40 29 15  3  2  1  2  0  0]
13775: [ 0  5 51 26 13  2  1  0  1  1]
13776: [ 0  6 54 26 13  0  1  0  0  0]
13777: [20  0  0 42 15 18  2  3  0  0]
13778: [ 0  4 38  2 19 22 13  1  1  0]
13779: [ 0  2 54 17 21  0  1  2  2  1]
13780: [22 17 56  0  0  2  1  2  0  0]
13781: [ 8 38 32 13  3  4  1  0  0  1]
13782: [ 0  0 48  2 16 20 12  1  0  1]
13783: [ 0  5 51 26 13  2  1  0  1  1]
13784: [32 15 20  8 12  5  0  3  4  1]
13785: [ 2 18 25 44  2  3  3  2  0  1]
13786: [ 0  6 54 26 13  0  1  0  0  0]
13787: [22 17 56  0  0  2  1  2  0  0]
13788: [27  4  5 25 17 19  1  0  2  0]
13789: [33 17 24  8  9  6

13975: [32 15 23  8 10  8  0  0  2  2]
13976: [ 0  0 51 16 22  0  2  4  3  2]
13977: [20  0  0 42 15 18  2  3  0  0]
13978: [20  0  0 42 15 18  2  3  0  0]
13979: [33 17 25  7  8  5  0  0  3  2]
13980: [ 8 40 29 15  3  2  1  2  0  0]
13981: [20  0  0 42 15 18  2  3  0  0]
13982: [33 17 24  8  9  6  0  0  1  2]
13983: [ 0  5 51 26 13  2  1  0  1  1]
13984: [ 0  3 61  3  1 14 10  8  0  0]
13985: [20  0  0 42 15 18  2  3  0  0]
13986: [33 17 25  7  8  5  0  0  3  2]
13987: [ 0  0 59 16 20  1  1  1  0  2]
13988: [ 1  5 22 43 25  3  0  0  1  0]
13989: [ 1  1 59  1  1 19 10  7  1  0]
13990: [ 0  2 54 17 21  0  1  2  2  1]
13991: [ 4  4 61  0  4 14  8  5  0  0]
13992: [20  0  0 42 15 18  2  3  0  0]
13993: [33 17 24  8  9  6  0  0  1  2]
13994: [ 8 40 29 15  3  2  1  2  0  0]
13995: [ 0  0 48  2 16 20 12  1  0  1]
13996: [ 5  9 59 15 12  0  0  0  0  0]
13997: [31  6  5 20 18 18  1  1  0  0]
13998: [33 17 24  8  9  6  0  0  1  2]
13999: [ 9 39 26 15  0  5  1  3  1  1]
14000: [ 0  5 51 26 13  2

14186: [ 0  5 51 26 13  2  1  0  1  1]
14187: [ 4  4 61  0  4 14  8  5  0  0]
14188: [27  4  5 25 17 19  1  0  2  0]
14189: [ 1  9 47 28 14  1  0  0  0  0]
14190: [20  1  0 41 16 18  0  1  2  1]
14191: [32 15 23  8 10  8  0  0  2  2]
14192: [ 0  6 55 26 13  0  0  0  0  0]
14193: [ 1 30 12 16 26  5  3  4  2  1]
14194: [ 8 39 28 15  1  5  1  2  1  0]
14195: [20  0  0 42 14 18  1  3  1  1]
14196: [ 0  4 38  2 19 22 13  1  1  0]
14197: [ 0  0 51 16 22  0  2  4  3  2]
14198: [22 17 56  0  0  2  1  2  0  0]
14199: [20  0  0 42 15 18  2  3  0  0]
14200: [32 17 23  7  8  4  1  4  2  2]
14201: [ 0  4 41  1 17 20 13  2  0  2]
14202: [22 17 56  0  0  2  1  2  0  0]
14203: [ 0 22 22 30  3  9  8  6  0  0]
14204: [ 8 40 30 14  2  4  0  0  1  1]
14205: [ 0  4 41  1 17 20 13  2  0  2]
14206: [ 0  2 54 17 21  0  1  2  2  1]
14207: [ 5  9 59 15 12  0  0  0  0  0]
14208: [ 1 30 15 17 25  3  2  4  2  1]
14209: [ 8 40 30 14  2  4  0  0  1  1]
14210: [20  0  0 42 15 18  2  3  0  0]
14211: [22 17 56  0  0  2

KeyboardInterrupt: 

Although it didn't converge to a single strategy, it seems to have settled in to some kind of pattern of strategies. A number of strategies keep showing up repeatedly in unchanged form which means they are most likely surviving through each generation as part of the top 16. For example [20,0,0,42,15,18,2,3,0,0] or [33,17,24,8,9,6,0,0,1,2] are particularly common ones. Because they are most likely surviving through the top 16 of every generation, one of these strategies may be a good bet for submission. However intuitively they don't seem particularly good, and I would like to try and make the model converge to one strategy.

Probably the best way of getting the model to converge is restricting all variation in the set of strategies fed into each generation to just the generated strategies. I.e sticking with the same random set, and creating a bank of good known strategies to serve in place of sampling the history.

First I will create a bank of 'known' strategies using the current form of the algorithm. I will change the train function to return the history.

In [57]:
#will train model for a specified number of generations, and output the highest performing strategy of the last model
def train(gensize,ngenerators,noise,number_of_generations,hist_sample_size,verbose = True):
    #generating some random strategies
    gen = random_generation(gensize)
    
    generators = next_generators(gen,ngenerators,gensize)
    if verbose:
        print('0: ' + str(generators[0]))
    
    past = np.array([generators[0]])
    gen = nextgen_from_generators(past,generators,ngenerators,gensize,noise)
    
    for i in range(1,number_of_generations):
        
        generators = next_generators(gen,ngenerators,gensize + np.minimum(i,hist_sample_size))
        if verbose:
            print(str(i) + ': ' + str(generators[0]))
            
        past = np.vstack((past, [generators[0].copy()]))
            
        gen = nextgen_from_generators(sample(past,hist_sample_size),generators,ngenerators,gensize,noise)
        
    return past
    

In [60]:
bank = train(128,16,40,1024,1024)

0: [12 12 15 10 12  8 10  6  8  7]
1: [ 0 11 15 11 16 17 12 11  5  2]
2: [ 8 17 17 18  1  7  9 11  6  6]
3: [ 9 17 17 21  1  7  9 12  4  3]
4: [ 6 16 20 17 19  5  6  0 10  1]
5: [ 4 15 23 24 20  0  3  1  8  2]
6: [ 3 13 24 26 19  0  1  0 11  3]
7: [ 1 18 26 25 11  2  2  3  7  5]
8: [ 1 18 29 28  8  3  0  2  7  4]
9: [ 0 17 32 28  4  2  2  3  8  4]
10: [ 3 21 34 28  6  1  2  0  5  0]
11: [ 3 21 34 28  6  1  2  0  5  0]
12: [ 4 20 28 29  7  5  1  0  3  3]
13: [ 0  0 40 18 20  5  5  1  0 11]
14: [ 0  0 41 26 20  4  4  2  2  1]
15: [ 0  0 39 33 15  0  3  2  6  2]
16: [ 2  0 39 23 17  8  5  3  2  1]
17: [ 2  3 42 22 19  4  4  0  2  2]
18: [ 2  3 42 22 19  4  4  0  2  2]
19: [ 8  3 48 18 18  0  2  3  0  0]
20: [ 3  6 48 22 15  0  1  3  1  1]
21: [ 4  5 46 16 20  2  4  2  0  1]
22: [ 4  5 46 16 20  2  4  2  0  1]
23: [ 2  5 32 23 21  7  4  3  0  3]
24: [ 8  0 33 22 21  8  5  1  2  0]
25: [ 1 19 38 30  7  1  0  0  3  1]
26: [ 1 22 38 27  6  2  2  0  1  1]
27: [ 4  6 42 22 17  4  3  1  0  1]
28

225: [ 0 35 19 26  1  1  9  5  4  0]
226: [ 0 35 19 26  1  1  9  5  4  0]
227: [ 1 36 19 23  3  1  5  6  5  1]
228: [ 1 36 19 23  3  1  5  6  5  1]
229: [ 0 33 20 26  0  6  7  7  0  1]
230: [ 0 33 20 26  0  6  7  7  0  1]
231: [ 1 34 12 29 10 10  1  3  0  0]
232: [ 1 34 13 27  3  9  5  3  4  1]
233: [ 3 36 19 25  0  4  5  4  2  2]
234: [ 4 36 21 24  0  0  7  4  4  0]
235: [ 4 36 21 24  0  0  7  4  4  0]
236: [ 1 37 21 27  1  0  5  4  3  1]
237: [ 2 37 22 23  2  1  6  3  2  2]
238: [ 2 37 22 23  2  1  6  3  2  2]
239: [ 2 36 22 25  2  2  6  2  3  0]
240: [ 2 34 19 25  4  5  6  3  2  0]
241: [ 2 34 19 25  4  5  6  3  2  0]
242: [ 4 34 18 25  3  4  5  3  2  2]
243: [ 1 38 13 23  5  6  8  2  1  3]
244: [ 1 38 13 23  5  6  8  2  1  3]
245: [ 2 36 22 25  2  2  6  2  3  0]
246: [ 1 38 13 23  5  6  8  2  1  3]
247: [ 1 38 13 23  5  6  8  2  1  3]
248: [ 1 34 12 29 10 10  1  3  0  0]
249: [ 1 34 12 29 10 10  1  3  0  0]
250: [ 3 36 20 26  0  1  6  5  3  0]
251: [ 5 34 13 28  4  4  6  2  2  2]
2

447: [19  2  4 37 25 12  1  0  0  0]
448: [19  2  4 37 25 12  1  0  0  0]
449: [19  2  4 37 25 12  1  0  0  0]
450: [17  1  5 38 26 11  0  2  0  0]
451: [17  1  5 38 26 11  0  2  0  0]
452: [17  1  5 38 26 11  0  2  0  0]
453: [17  1  5 38 26 11  0  2  0  0]
454: [21  3  6 32 22 15  0  0  1  0]
455: [17  1  5 38 26 11  0  2  0  0]
456: [21  3  6 32 22 15  0  0  1  0]
457: [21  3  6 32 22 15  0  0  1  0]
458: [21  3  6 32 22 12  2  0  1  1]
459: [20  4  0 31 25 15  3  0  1  1]
460: [21  3  0 32 25 15  4  0  0  0]
461: [21  3  0 32 25 15  4  0  0  0]
462: [21  3  0 32 25 15  4  0  0  0]
463: [17  1  5 38 26 11  0  2  0  0]
464: [21  3  6 32 22 12  2  0  1  1]
465: [21  3  6 32 22 12  2  0  1  1]
466: [19  2  4 37 25 12  1  0  0  0]
467: [20  4  0 31 25 15  3  0  1  1]
468: [17  1  5 38 26 11  0  2  0  0]
469: [21  3  6 32 22 15  0  0  1  0]
470: [21  3  0 32 25 15  4  0  0  0]
471: [18  4  1 33 25 14  3  0  2  0]
472: [21  3  6 32 22 12  2  0  1  1]
473: [21  3  6 32 22 12  2  0  1  1]
4

669: [ 3 13 26 41  6  4  4  2  1  0]
670: [ 1 10 28 44 12  1  1  1  0  2]
671: [ 0 16 23 44  6  0  6  3  2  0]
672: [ 2 10 24 44 11  1  4  2  2  0]
673: [ 3 13 26 41  6  4  4  2  1  0]
674: [ 1 10 28 44 12  1  1  1  0  2]
675: [ 0 13 23 42  7  1  7  3  2  2]
676: [ 0 13 23 42  7  1  7  3  2  2]
677: [ 2 10 24 44 11  1  4  2  2  0]
678: [ 3 13 26 41  6  4  4  2  1  0]
679: [ 0 16 23 44  6  0  6  3  2  0]
680: [ 0 13 23 42  7  1  7  3  2  2]
681: [ 1 14 26 43  3  4  3  3  1  2]
682: [ 3 13 26 41  6  4  4  2  1  0]
683: [ 0 14 27 44  6  2  3  2  2  0]
684: [ 0 14 27 44  6  2  3  2  2  0]
685: [ 1 14 24 40  8  1  6  2  2  2]
686: [ 0 14 27 44  6  2  3  2  2  0]
687: [ 1  9 24 44 10  2  4  3  2  1]
688: [ 0 14 27 44  6  2  3  2  2  0]
689: [ 1  9 24 44 10  2  4  3  2  1]
690: [ 1 11 25 45  8  3  3  3  1  0]
691: [ 0 13 30 45  8  0  1  1  0  2]
692: [ 1 10 28 45 11  2  1  1  0  1]
693: [ 3 16 28 40  4  3  2  3  1  0]
694: [ 0 13 30 45  8  0  1  1  0  2]
695: [ 1 14 24 40  8  1  6  2  2  2]
6

891: [26 16 40  7  7  4  0  0  0  0]
892: [24 14 42  0  0  8  1  4  3  4]
893: [24 17 45  0  1  6  3  3  1  0]
894: [23 19 44  5  4  1  1  1  1  1]
895: [24 17 45  0  1  6  3  3  1  0]
896: [26 16 34  6  7  0  1  4  3  3]
897: [26 16 34  6  7  0  1  4  3  3]
898: [26 16 34  6  7  0  1  4  3  3]
899: [26 16 34  6  7  0  1  4  3  3]
900: [23 20 41  9  5  1  0  0  1  0]
901: [25 21 35  1  6  3  6  1  1  1]
902: [26 16 34  6  7  0  1  4  3  3]
903: [26 16 40  7  7  4  0  0  0  0]
904: [24 17 45  0  1  6  3  3  1  0]
905: [23 19 44  5  4  1  1  1  1  1]
906: [24 17 45  0  1  6  3  3  1  0]
907: [26 16 34  6  7  0  1  4  3  3]
908: [23 20 41  9  5  1  0  0  1  0]
909: [24 14 42  0  0  8  1  4  3  4]
910: [23 17 36  2  3 11  1  1  4  2]
911: [23 20 41  9  5  1  0  0  1  0]
912: [24 17 45  0  1  6  3  3  1  0]
913: [23 19 44  5  4  1  1  1  1  1]
914: [25 21 35  1  6  3  6  1  1  1]
915: [24 14 42  0  0  8  1  4  3  4]
916: [24 17 45  0  1  6  3  3  1  0]
917: [23 17 36  2  3 11  1  1  4  2]
9


Adding some obvious strategies:

In [19]:
bank = np.vstack((bank,[[10,10,10,10,10,10,10,10,10,10],[33,33,34,0,0,0,0,0,0,0],[33,34,33,0,0,0,0,0,0,0],[34,33,33,0,0,0,0,0,0,0],[0,35,0,25,20,20,0,0,0,0]]))

Adding some random strategies

In [49]:
bank = np.vstack((bank,random_generation(400)))

I've just realised a very sensible thing to do is to seperate the generation from the collection of strategies being scored against, which will likely massively reduce computation time and improve performance of the algorithm. i.e there should be a separation between the strategies being trained on and the evolving strategies otherwise an awful lot of data is being copied over and over that doesn't need to be. Furthermore I feel that adding a bunch of non-generated strategies to every generation disrupts the natural progression of the algorithm, and almost defeats the point of it.

In [90]:

def train(strat_bank,gensize,ngenerators,noise,number_of_generations,verbose = True):
    
    gen = random_generation(gensize)
    
    generators = next_generators(strat_bank,gen,ngenerators,gensize)
    if verbose:
        print('0: ' + str(generators[0]))
    
    
    gen = nextgen_from_generators(generators,ngenerators,gensize,noise)
    
    for i in range(1,number_of_generations):
        
        generators = next_generators(strat_bank,gen,ngenerators,gensize)
        if verbose:
            print(str(i) + ': ' + str(generators[0]))
            
        
            
        gen = nextgen_from_generators(generators,ngenerators,gensize,noise)
        
    return next_generators(strat_bank,gen,ngenerators,gensize)[0]
    

In [68]:
#will train model until manual interrupt
def train_until_interrupt(strat_bank,gensize,ngenerators,noise,verbose = True):
    #generating some random strategies
    gen = random_generation(gensize)
    
    generators = next_generators(strat_bank,gen,ngenerators,gensize)
    if verbose:
        print('0: ' + str(generators[0]))
    
    
    gen = nextgen_from_generators(generators,ngenerators,gensize,noise)
    
    i = 0
    while True:
        i += 1
        generators = next_generators(strat_bank,gen,ngenerators,gensize)
        if verbose:
            print(str(i) + ': ' + str(generators[0]))
            
        
        
            
        gen = nextgen_from_generators(generators,ngenerators,gensize,noise)
        
    return 0
    

In [67]:
def nextgen_from_generators(generators,ngen,gensize,noise,inject_random = False):
    nextgen = np.empty((gensize,10),dtype = np.uint8)
    
    if inject_random:
        factor = gensize // (2*ngen)
        for i in range(0,ngen):
            nextgen[i*factor] = generators[i]
            for j in range(1,factor):
                nextgen[i*factor + j] = mutate(generators[i],noise)
                
        nextgen[(gensize//2):gensize] = random_generation(gensize//2)
            
    else:
        factor = gensize // ngen
        for i in range(0,ngen):
            nextgen[i*factor] = generators[i]
            for j in range(1,factor):
                nextgen[i*factor + j] = mutate(generators[i],noise)
    
    return nextgen.copy()

In [66]:
def next_generators(strat_bank,generation,ngen,gensize):
    scores = np.empty(gensize,dtype = np.uint64)
    for i in range(0,gensize):
        scores[i] = generation_score(generation[i],strat_bank)
    
    indexes = (-scores).argsort()[:ngen]
    return generation[indexes].copy()   

In [56]:
train_until_interrupt(bank,128,16,20)

0: [14  9 13 15 13  7  6  5 11  7]
1: [17 10 15 18 15 10  4  1  6  4]
2: [17 10 15 18 15 10  4  1  6  4]
3: [19 11 16 14 16  9  2  5  5  3]
4: [18 12  9 15 17 16  0  5  3  5]
5: [18  3  2 20 16 17 12  5  2  5]
6: [17 10  9 16 18 20  0  1  5  4]
7: [19  5  4 18 22 18  0  6  5  3]
8: [19  5  4 18 22 18  0  6  5  3]
9: [18  4  5 19 19 19  7  5  1  3]
10: [18  4  5 19 19 19  7  5  1  3]
11: [18  4  5 19 19 19  7  5  1  3]
12: [19  4  7 19 25 19  2  1  3  1]
13: [19  4  7 19 25 19  2  1  3  1]
14: [19  4  7 19 25 19  2  1  3  1]
15: [19  4  7 19 25 19  2  1  3  1]
16: [16  0  5 24 22 20  4  1  5  3]
17: [16  0  5 24 22 20  4  1  5  3]
18: [16  0  5 24 22 20  4  1  5  3]
19: [21  2  4 20 20 20  0  5  4  4]
20: [21  2  4 20 20 20  0  5  4  4]
21: [21  2  4 20 20 20  0  5  4  4]
22: [21  2  4 20 20 20  0  5  4  4]
23: [21  2  4 20 20 20  0  5  4  4]
24: [21  2  4 20 20 20  0  5  4  4]
25: [18  2  2 17 32 22  4  2  1  0]
26: [16  3  3 19 35 18  2  1  2  1]
27: [16  3  3 19 35 18  2  1  2  1]
28

225: [16  0 17 24 33  7  0  0  1  2]
226: [16  0 17 24 33  7  0  0  1  2]
227: [16  0 17 24 33  7  0  0  1  2]
228: [16  0 17 24 33  7  0  0  1  2]
229: [16  0 17 24 33  7  0  0  1  2]
230: [16  0 17 24 33  7  0  0  1  2]
231: [16  0 17 24 33  7  0  0  1  2]
232: [16  0 17 24 33  7  0  0  1  2]
233: [16  0 17 24 33  7  0  0  1  2]
234: [16  0 17 24 33  7  0  0  1  2]
235: [16  0 17 24 33  7  0  0  1  2]
236: [16  0 17 24 33  7  0  0  1  2]
237: [16  0 17 24 33  7  0  0  1  2]
238: [16  0 17 24 33  7  0  0  1  2]
239: [16  0 17 24 33  7  0  0  1  2]
240: [16  0 17 24 33  7  0  0  1  2]
241: [16  0 17 24 33  7  0  0  1  2]
242: [16  0 17 24 33  7  0  0  1  2]
243: [16  0 17 24 33  7  0  0  1  2]
244: [16  0 17 24 33  7  0  0  1  2]
245: [16  0 17 24 33  7  0  0  1  2]
246: [16  0 17 24 33  7  0  0  1  2]
247: [16  0 17 24 33  7  0  0  1  2]
248: [16  0 17 24 33  7  0  0  1  2]
249: [16  0 17 24 33  7  0  0  1  2]
250: [16  0 17 24 33  7  0  0  1  2]
251: [16  0 17 24 33  7  0  0  1  2]
2

447: [16  2 17 19 33  8  2  1  2  0]
448: [16  2 17 19 33  8  2  1  2  0]
449: [16  2 17 19 33  8  2  1  2  0]
450: [16  2 17 19 33  8  2  1  2  0]
451: [16  2 17 19 33  8  2  1  2  0]
452: [16  2 17 19 33  8  2  1  2  0]
453: [16  2 17 19 33  8  2  1  2  0]
454: [16  2 17 19 33  8  2  1  2  0]
455: [16  2 17 19 33  8  2  1  2  0]
456: [16  2 17 19 33  8  2  1  2  0]
457: [16  2 17 19 33  8  2  1  2  0]
458: [16  2 17 19 33  8  2  1  2  0]
459: [16  2 17 19 33  8  2  1  2  0]
460: [16  2 17 19 33  8  2  1  2  0]
461: [16  2 17 19 33  8  2  1  2  0]
462: [16  2 17 19 33  8  2  1  2  0]
463: [16  2 17 19 33  8  2  1  2  0]
464: [16  2 17 19 33  8  2  1  2  0]
465: [16  2 17 19 33  8  2  1  2  0]
466: [16  2 17 19 33  8  2  1  2  0]
467: [16  2 17 19 33  8  2  1  2  0]
468: [16  2 17 19 33  8  2  1  2  0]
469: [16  2 17 19 33  8  2  1  2  0]
470: [16  2 17 19 33  8  2  1  2  0]
471: [16  2 17 19 33  8  2  1  2  0]
472: [16  2 17 19 33  8  2  1  2  0]
473: [16  2 17 19 33  8  2  1  2  0]
4

669: [16  2 18 21 33  7  2  0  0  1]
670: [16  2 18 21 33  7  2  0  0  1]
671: [16  2 18 21 33  7  2  0  0  1]
672: [16  2 18 21 33  7  2  0  0  1]
673: [16  2 18 21 33  7  2  0  0  1]
674: [16  2 18 21 33  7  2  0  0  1]
675: [16  2 18 21 33  7  2  0  0  1]
676: [16  2 18 21 33  7  2  0  0  1]
677: [16  2 18 21 33  7  2  0  0  1]
678: [16  2 18 21 33  7  2  0  0  1]
679: [16  2 18 21 33  7  2  0  0  1]
680: [16  2 18 21 33  7  2  0  0  1]
681: [16  2 18 21 33  7  2  0  0  1]
682: [16  2 18 21 33  7  2  0  0  1]
683: [16  2 18 21 33  7  2  0  0  1]
684: [16  2 18 21 33  7  2  0  0  1]
685: [16  2 18 21 33  7  2  0  0  1]
686: [16  2 18 21 33  7  2  0  0  1]
687: [16  2 18 21 33  7  2  0  0  1]
688: [16  2 18 21 33  7  2  0  0  1]
689: [16  2 18 21 33  7  2  0  0  1]
690: [16  2 18 21 33  7  2  0  0  1]
691: [16  2 18 21 33  7  2  0  0  1]
692: [16  2 18 21 33  7  2  0  0  1]
693: [16  2 18 21 33  7  2  0  0  1]
694: [16  2 18 21 33  7  2  0  0  1]
695: [16  2 18 21 33  7  2  0  0  1]
6

891: [16  2 18 21 33  7  2  0  0  1]
892: [16  2 18 21 33  7  2  0  0  1]
893: [16  2 18 21 33  7  2  0  0  1]
894: [16  2 18 21 33  7  2  0  0  1]
895: [16  2 18 21 33  7  2  0  0  1]
896: [16  2 18 21 33  7  2  0  0  1]
897: [16  2 18 21 33  7  2  0  0  1]
898: [16  2 18 21 33  7  2  0  0  1]
899: [16  2 18 21 33  7  2  0  0  1]
900: [16  2 18 21 33  7  2  0  0  1]
901: [16  2 18 21 33  7  2  0  0  1]
902: [16  2 18 21 33  7  2  0  0  1]
903: [16  2 18 21 33  7  2  0  0  1]
904: [16  2 18 21 33  7  2  0  0  1]
905: [16  2 18 21 33  7  2  0  0  1]
906: [16  2 18 21 33  7  2  0  0  1]
907: [16  2 18 21 33  7  2  0  0  1]
908: [16  2 18 21 33  7  2  0  0  1]
909: [16  2 18 21 33  7  2  0  0  1]
910: [16  2 18 21 33  7  2  0  0  1]
911: [16  2 18 21 33  7  2  0  0  1]
912: [16  2 18 21 33  7  2  0  0  1]
913: [16  2 18 21 33  7  2  0  0  1]
914: [16  2 18 21 33  7  2  0  0  1]
915: [16  2 18 21 33  7  2  0  0  1]
916: [16  2 18 21 33  7  2  0  0  1]
917: [16  2 18 21 33  7  2  0  0  1]
9

KeyboardInterrupt: 

I think perhaps a better way of doing it is constructing a strategy bank with all the types of strategies that I can think of and then training on that, i.e constructing what I think might be a good representation of the collection of Jane Streeters' strategies I will ultimately be up against.

In [62]:
#creates and fills an array of size 10, in only specified indices, that sums to 100, using random.choice for randomness, returns this array
def randomfill(indexes = [0,1,2,3,4,5,6,7,8,9]):
    arr = np.zeros(10,dtype = np.uint8)
    
    for i in range(0,100):
        arr[random.choice(indexes)] += 1
        
    return arr

In [63]:
#creates a generation of size 'gensize' of random strategies
def strategy_bank(size):
    bank = np.empty((size,10),dtype = np.uint8)
    idxs = [[0,1,2],[1,2,3],[2,3,4],[0,3,4,5],[0,4,5,6],[1,3,4,5],[1,4,5,6],[0,3,5,6,7],[2,4,5,6],[2,3,4,5],[2,5,6,7],[0,1,2,3,4,5,6,7,8,9]]
    
    for i in range(0,size):
        bank[i] = randomfill(idxs[np.random.choice(12)])
    
    return bank

In [65]:
bank = strategy_bank(512)

In [72]:
train_until_interrupt(bank,128,16,20)

0: [13 11  6  8  9 20 10  6  8  9]
1: [10 11  4 14 23  8 10 14  2  4]
2: [ 9 10  5 15 27  6  9 13  3  3]
3: [ 7  9  4 19 29  8  7 11  4  2]
4: [ 6  7  3 24 35  5  6 11  3  0]
5: [ 3  9  6 27 35  7  1  9  2  1]
6: [ 3  9  6 27 35  7  1  9  2  1]
7: [ 0  3 12 35 32  1  3  8  3  3]
8: [ 0  4 14 37 32  1  1  6  5  0]
9: [ 0  1 19 38 32  1  0  6  2  1]
10: [ 1  2 16 38 30  1  3  3  3  3]
11: [ 1  2 17 39 33  0  3  2  1  2]
12: [ 2  1 17 37 33  1  1  4  3  1]
13: [ 1  0 26 36 31  1  0  1  3  1]
14: [ 0  1 27 36 31  1  2  0  2  0]
15: [ 0  1 28 38 27  0  2  2  1  1]
16: [ 0  1 32 38 23  2  2  1  1  0]
17: [ 0  1 33 38 24  1  1  2  0  0]
18: [ 0  2 34 35 24  2  1  2  0  0]
19: [ 2  1 35 28 28  1  1  1  2  1]
20: [ 1  1 36 28 28  1  1  1  1  2]
21: [ 1  1 36 28 28  1  1  1  1  2]
22: [ 1  1 36 28 28  1  1  1  1  2]
23: [ 1  1 38 29 26  1  2  1  1  0]
24: [ 1  1 38 29 26  1  2  1  1  0]
25: [ 1  1 38 29 26  1  2  1  1  0]
26: [ 1  1 38 29 26  1  2  1  1  0]
27: [ 1  2 41 29 23  1  1  1  1  0]
28

225: [ 1  1 41 26 27  1  1  1  1  0]
226: [ 1  1 41 26 27  1  1  1  1  0]
227: [ 1  1 41 26 27  1  1  1  1  0]
228: [ 1  1 41 26 27  1  1  1  1  0]
229: [ 1  1 41 26 27  1  1  1  1  0]
230: [ 1  1 41 26 27  1  1  1  1  0]
231: [ 1  1 41 26 27  1  1  1  1  0]
232: [ 1  1 41 26 27  1  1  1  1  0]
233: [ 1  1 41 26 27  1  1  1  1  0]
234: [ 1  1 41 26 27  1  1  1  1  0]
235: [ 1  1 41 26 27  1  1  1  1  0]
236: [ 1  1 41 26 27  1  1  1  1  0]
237: [ 1  1 41 26 27  1  1  1  1  0]
238: [ 1  1 41 26 27  1  1  1  1  0]
239: [ 1  1 41 26 27  1  1  1  1  0]
240: [ 1  1 41 26 27  1  1  1  1  0]
241: [ 1  1 41 26 27  1  1  1  1  0]
242: [ 1  1 41 26 27  1  1  1  1  0]
243: [ 1  1 41 26 27  1  1  1  1  0]
244: [ 1  1 41 26 27  1  1  1  1  0]
245: [ 1  1 41 26 27  1  1  1  1  0]
246: [ 1  1 41 26 27  1  1  1  1  0]
247: [ 1  1 41 26 27  1  1  1  1  0]
248: [ 1  1 41 26 27  1  1  1  1  0]
249: [ 1  1 41 26 27  1  1  1  1  0]
250: [ 1  1 41 26 27  1  1  1  1  0]
251: [ 1  1 41 26 27  1  1  1  1  0]
2

447: [ 1  1 41 26 27  1  1  1  1  0]
448: [ 1  1 41 26 27  1  1  1  1  0]
449: [ 1  1 41 26 27  1  1  1  1  0]
450: [ 1  1 41 26 27  1  1  1  1  0]
451: [ 1  1 41 26 27  1  1  1  1  0]
452: [ 1  1 41 26 27  1  1  1  1  0]
453: [ 1  1 41 26 27  1  1  1  1  0]
454: [ 1  1 41 26 27  1  1  1  1  0]
455: [ 1  1 41 26 27  1  1  1  1  0]
456: [ 1  1 41 26 27  1  1  1  1  0]
457: [ 1  1 41 26 27  1  1  1  1  0]
458: [ 1  1 41 26 27  1  1  1  1  0]
459: [ 1  1 41 26 27  1  1  1  1  0]
460: [ 1  1 41 26 27  1  1  1  1  0]
461: [ 1  1 41 26 27  1  1  1  1  0]


KeyboardInterrupt: 

This seems like a success. It has very quickly converged to a particular strategy focussing on the 3rd, 4th and 5th castle. Putting 1s in the castles that would otherwise be 0 is something I hadn't initially thought. It certainly makes a lot of sense and provides an improvement over just having 0s in those castles, as it can catch a win whenever there is a 0 in the castle. I think any human made strategy is likely to have quite a few 0 soldier castles depending on how much thought they've put into it.

Lets try again with a larger strategy bank

In [73]:
bank = strategy_bank(1024)


In [74]:
train_until_interrupt(bank,128,16,20)

0: [11  8  9 20 10  7  8  9 13  5]
1: [13  6  7 23  8  8  7  9 13  6]
2: [10  8  9  4  5 26 10 11  5 12]
3: [12  3 12  6 13 28  5  7  6  8]
4: [ 4 12  6  2 18 29  4 13 11  1]
5: [11  8  3  4 30 28  4  8  0  4]
6: [10  6  4  2 30 29  4  9  1  5]
7: [ 7  6  6  4 35 26  3  9  2  2]
8: [13  1  1  2 35 31  3  8  2  4]
9: [11  2  2  7 38 31  1  5  1  2]
10: [11  2  2  7 38 31  1  5  1  2]
11: [11  1  1  9 41 29  2  1  3  2]
12: [11  1  1  9 41 29  2  1  3  2]
13: [11  1  3 10 39 30  1  2  1  2]
14: [11  1  3 10 39 30  1  2  1  2]
15: [10  1  1 11 40 34  1  0  1  1]
16: [10  1  1 11 40 34  1  0  1  1]
17: [10  1  1 11 40 34  1  0  1  1]
18: [10  1  1 11 40 34  1  0  1  1]
19: [10  1  1 11 40 34  1  0  1  1]
20: [10  1  1 11 40 34  1  0  1  1]
21: [10  1  1 11 40 34  1  0  1  1]
22: [10  1  1 11 40 34  1  0  1  1]
23: [10  1  1 11 40 34  1  0  1  1]
24: [12  1  1 12 38 31  1  0  3  1]
25: [12  1  1 12 38 33  1  0  1  1]
26: [12  1  1 12 38 33  1  0  1  1]
27: [12  1  1 12 38 33  1  0  1  1]
28

KeyboardInterrupt: 

In [80]:
generation_score([12,1,1,11,40,32,1,0,1,1],bank)

38981

In [81]:
generation_score([1,1,41,26,27,1,1,1,1,0],bank)

51044

In [82]:
train_until_interrupt(bank,256,16,40)

0: [10  8 13 11 19  8  6 10  8  7]
1: [ 0  7 20 25  6  3 15 11  8  5]
2: [ 1  4 20 30  7  6 15  9  5  3]
3: [ 6  4 35  9  7 19  0 11  4  5]
4: [ 7  3 31 28 12  8  5  5  0  1]
5: [ 3  3 34 30 11  5  5  5  1  3]
6: [ 6  3 38 29 11  1  3  3  3  3]
7: [ 1  4 36 34 14  3  3  2  1  2]
8: [ 1  4 38 33 16  2  2  2  1  1]
9: [ 1  1 44 33 13  4  1  2  1  0]
10: [ 1  1 44 33 13  4  1  2  1  0]
11: [ 2  1 39 33 18  1  3  1  1  1]
12: [ 1  1 43 32 16  4  1  1  1  0]
13: [ 2  2 44 33 15  1  1  1  1  0]
14: [ 2  2 44 33 15  1  1  1  1  0]
15: [ 2  2 44 33 15  1  1  1  1  0]
16: [ 2  2 44 33 15  1  1  1  1  0]
17: [ 1  1 41 36 15  2  1  1  1  1]
18: [ 1  1 42 35 16  1  1  2  1  0]
19: [ 1  1 42 35 16  1  1  2  1  0]
20: [ 1  1 42 35 16  1  1  2  1  0]
21: [ 1  1 42 35 16  1  1  2  1  0]
22: [ 1  1 42 35 16  1  1  2  1  0]
23: [ 1  1 42 35 16  1  1  2  1  0]
24: [ 1  1 42 35 16  1  1  2  1  0]
25: [ 1  1 42 35 16  1  1  2  1  0]
26: [ 1  1 42 35 16  1  1  2  1  0]
27: [ 1  1 42 35 16  1  1  2  1  0]
28

KeyboardInterrupt: 

In [83]:
generation_score([1,1,38,30,26,1,1,1,1,0],bank)

51197

In [84]:
train_until_interrupt(bank,256,16,5)

0: [ 4 10 11 10  7 20  8 15  7  8]
1: [ 2 13 14 20  9  3 15 11  7  6]
2: [ 1 11 11  8  8 24  8 14  6  9]
3: [ 3 14 14 24  8  1 16 10  4  6]
4: [ 1  8 10 10  7 28  7 13  6 10]
5: [10  3  6  9 14 31  9  7 10  1]
6: [ 3 13 15 30  4  2 14 10  4  5]
7: [ 3 13 16 32  5  2 14  9  3  3]
8: [ 2 13 16 34  5  2 13  9  4  2]
9: [ 3 13 15 36  3  2 15 10  1  2]
10: [ 3 14 15 37  3  1 13 11  2  1]
11: [ 1 14 16 39  2  2 15  6  2  3]
12: [ 1 15 16 41  2  3 15  3  1  3]
13: [ 1 20 16 40  2  1 10  7  2  1]
14: [ 4 14 22 40  4  2  9  1  2  2]
15: [ 1 24 16 41  1  2  9  4  2  0]
16: [ 3 15 24 39  4  1  9  1  2  2]
17: [ 2 15 25 41  2  2  9  1  1  2]
18: [ 1 15 27 40  5  1  4  2  3  2]
19: [ 3 15 29 38  2  2  3  2  4  2]
20: [ 1 13 32 40  3  2  3  1  4  1]
21: [ 2 13 33 40  3  2  4  1  2  0]
22: [ 3 14 35 39  2  1  1  1  3  1]
23: [ 2 13 37 39  1  0  4  1  2  1]
24: [ 2 14 38 39  1  0  3  1  2  0]
25: [ 2 13 39 40  1  1  2  1  1  0]
26: [ 1 15 39 37  2  1  2  1  1  1]
27: [ 1 15 39 37  2  1  2  1  1  1]
28

225: [ 1 29 38 27  1  1  1  1  1  0]
226: [ 1 29 38 27  1  1  1  1  1  0]
227: [ 1 29 38 27  1  1  1  1  1  0]
228: [ 1 29 38 27  1  1  1  1  1  0]
229: [ 1 29 38 27  1  1  1  1  1  0]
230: [ 1 29 38 27  1  1  1  1  1  0]
231: [ 1 29 38 27  1  1  1  1  1  0]
232: [ 1 29 38 27  1  1  1  1  1  0]
233: [ 1 29 38 27  1  1  1  1  1  0]
234: [ 1 29 38 27  1  1  1  1  1  0]
235: [ 1 29 38 27  1  1  1  1  1  0]
236: [ 1 29 38 27  1  1  1  1  1  0]
237: [ 1 29 38 27  1  1  1  1  1  0]
238: [ 1 29 38 27  1  1  1  1  1  0]
239: [ 1 29 38 27  1  1  1  1  1  0]
240: [ 1 29 38 27  1  1  1  1  1  0]
241: [ 1 29 38 27  1  1  1  1  1  0]
242: [ 1 29 38 27  1  1  1  1  1  0]
243: [ 1 29 38 27  1  1  1  1  1  0]
244: [ 1 29 38 27  1  1  1  1  1  0]
245: [ 1 29 38 27  1  1  1  1  1  0]
246: [ 1 29 38 27  1  1  1  1  1  0]
247: [ 1 29 38 27  1  1  1  1  1  0]
248: [ 1 29 38 27  1  1  1  1  1  0]
249: [ 1 29 38 27  1  1  1  1  1  0]
250: [ 1 29 38 27  1  1  1  1  1  0]
251: [ 1 29 38 27  1  1  1  1  1  0]
2

447: [ 1 29 38 27  1  1  1  1  1  0]
448: [ 1 29 38 27  1  1  1  1  1  0]
449: [ 1 29 38 27  1  1  1  1  1  0]
450: [ 1 29 38 27  1  1  1  1  1  0]
451: [ 1 29 38 27  1  1  1  1  1  0]
452: [ 1 29 38 27  1  1  1  1  1  0]
453: [ 1 29 38 27  1  1  1  1  1  0]
454: [ 1 29 38 27  1  1  1  1  1  0]
455: [ 1 29 38 27  1  1  1  1  1  0]
456: [ 1 29 38 27  1  1  1  1  1  0]
457: [ 1 29 38 27  1  1  1  1  1  0]
458: [ 1 29 38 27  1  1  1  1  1  0]
459: [ 1 29 38 27  1  1  1  1  1  0]
460: [ 1 29 38 27  1  1  1  1  1  0]
461: [ 1 29 38 27  1  1  1  1  1  0]
462: [ 1 29 38 27  1  1  1  1  1  0]
463: [ 1 29 38 27  1  1  1  1  1  0]
464: [ 1 29 38 27  1  1  1  1  1  0]
465: [ 1 29 38 27  1  1  1  1  1  0]
466: [ 1 29 38 27  1  1  1  1  1  0]
467: [ 1 29 38 27  1  1  1  1  1  0]
468: [ 1 29 38 27  1  1  1  1  1  0]
469: [ 1 29 38 27  1  1  1  1  1  0]
470: [ 1 29 38 27  1  1  1  1  1  0]
471: [ 1 29 38 27  1  1  1  1  1  0]
472: [ 1 29 38 27  1  1  1  1  1  0]
473: [ 1 29 38 27  1  1  1  1  1  0]
4

KeyboardInterrupt: 

In [85]:
generation_score([1,29,38,27,1,1,1,1,1,0],bank)

51833

In [88]:
numwins([1,29,38,27,1,1,1,1,1,0],bank)

964

From these experiments it looks like this algorithm converges to high performing strategies, however there is a lot of variation between each time the algorithm is run, and also I imagine between different strategy banks. The obvious next step is to generate a number of strategies from a number of different strategy banks, and then choose the highest performing one scored against a larger unseen set of strategies.

Note running get_best_strat could take a day or more

In [105]:
#note 5 must divide number_of_strats for this to work properly
#also note it takes about a minute per strategy to generate
def generate_strategies(number_of_strats):
    strats = np.empty((number_of_strats,10),dtype = np.uint8)
    nbanks = number_of_strats // 5
    for i in range(0,nbanks):
        bank = strategy_bank(512)
        for j in range(0,5):
            strats[5*i + j] = train(bank,256,16,5,100,verbose = False)
            print(str(5*i + j) + ': ' + str(strats[5*i + j]))
            
    return strats

In [101]:
def get_best_strat(testbanksize = 4096,samplesize = 200):
    good_strategies = generate_strategies(samplesize)
    bank = strategy_bank(testbanksize)
    return next_generators(bank,good_strategies,1,samplesize)

In [107]:
get_best_strat(samplesize = 600) 

0: [ 1  1 20 39 34  1  1  1  1  1]
1: [35 13 13  1  1 33  1  1  1  1]
2: [ 1  1 24 37 32  2  1  1  1  0]
3: [ 1  1 50  1  1 37  1  1  6  1]
4: [ 1  1 41 32 21  1  1  1  1  0]
5: [40 10  1  1 43  1  1  1  1  1]
6: [ 1 11 12 39  1 32  1  1  1  1]
7: [45  1  1  4 44  1  1  1  1  1]
8: [29 30 37  1  1  1  1  0  0  0]
9: [ 1  1 13 39 10 32  1  1  1  1]
10: [ 1  1 35 31 28  1  1  1  1  0]
11: [ 1  1 13 38 12 31  1  1  1  1]
12: [ 1 36 19 39  1  1  1  1  1  0]
13: [14 13 39  1  1 28  1  1  1  1]
14: [ 1  1 35 31 28  1  1  1  1  0]
15: [29 31 36  1  1  1  1  0  0  0]
16: [ 1 31 38 27  1  1  1  0  0  0]
17: [ 1 31 38 27  1  1  1  0  0  0]
18: [ 1  1 39 26 29  1  1  1  1  0]
19: [29 31 36  1  1  1  1  0  0  0]
20: [15 36 15  1  1  1 28  1  1  1]
21: [ 1  1 39 26 29  1  1  1  1  0]
22: [11  1  1 36 32 15  1  1  1  1]
23: [ 1 15  1 14 34 31  1  1  1  1]
24: [ 1  1 43  7 12 32  1  1  1  1]
25: [ 1 30 37 29  1  1  1  0  0  0]
26: [ 1 30 37 29  1  1  1  0  0  0]
27: [14  1  1 44  1 35  1  1  1  1]
28

225: [ 1  1 12 39 12 31  1  1  1  1]
226: [ 1  1 39 26 29  1  1  1  1  0]
227: [ 1 14  1 14 36 30  1  1  1  1]
228: [ 1  1 13  1 36 30 15  1  1  1]
229: [ 1  1 18 40 35  1  1  1  1  1]
230: [ 1  1 37 31 26  1  1  1  1  0]
231: [ 1 11  9 40  1 34  1  1  1  1]
232: [15  1  1 43  1 35  1  1  1  1]
233: [38 13  9  1  1 34  1  1  1  1]
234: [ 1  1 37 31 26  1  1  1  1  0]
235: [ 1  1 38 26 30  1  1  1  1  0]
236: [ 1  1 40 35 19  1  1  1  1  0]
237: [ 1  1 38 26 30  1  1  1  1  0]
238: [ 1 12  1  1 36 33 14  0  1  1]
239: [ 1  1 40 27 27  1  1  1  1  0]
240: [ 1 19 41 34  1  1  1  1  1  0]
241: [ 1  1 41  1 11 32 10  1  1  1]
242: [ 1  1 14 38 12 30  1  1  1  1]
243: [ 1 12 14 38  1 30  1  1  1  1]
244: [ 1 21 41 32  1  1  1  1  1  0]
245: [ 1 11  1 38 13 32  1  1  1  1]
246: [ 1 13  1  1 37 30 14  1  1  1]
247: [ 1 29 37 29  1  1  1  1  0  0]
248: [ 1  1 39 29 26  1  1  1  1  0]
249: [ 1 13  1  1 37 33 12  0  1  1]
250: [15  1  1 44  1  1 34  1  1  1]
251: [ 1  1 38 27 29  1  1  1  1  0]
2

447: [ 1  1 39 28 27  1  1  1  1  0]
448: [15  1  1 41  1 37  1  1  1  1]
449: [ 1  1 39 28 27  1  1  1  1  0]
450: [ 1  1 40 25 29  1  1  1  1  0]
451: [ 1  1 39 25 30  1  1  1  1  0]
452: [ 1  1 39 25 30  1  1  1  1  0]
453: [ 1 11  1 39 11 33  1  1  1  1]
454: [ 1  1 39 25 30  1  1  1  1  0]
455: [ 1  1 39 28 27  1  1  1  1  0]
456: [ 1  1 39 24 31  1  1  1  1  0]
457: [ 1  1 39 24 31  1  1  1  1  0]
458: [ 1  1 12 36 13 33  1  1  1  1]
459: [ 1  1 39  1 12 31 12  1  1  1]
460: [ 1  1 15 43 35  1  1  1  1  1]
461: [ 1  1 15 43 35  1  1  1  1  1]
462: [ 1  1 21 41 31  1  1  1  1  1]
463: [ 1  1 37 29 28  1  1  1  1  0]
464: [ 1  1 41  1  1 36 16  1  1  1]
465: [ 1 31 38 27  1  1  1  0  0  0]
466: [13  1  1 43  1 37  1  1  1  1]
467: [ 1 31 38 27  1  1  1  0  0  0]
468: [ 1  1 38 27 29  1  1  1  1  0]
469: [ 1  1 15  1 39 12 28  1  1  1]
470: [36 13 14  1  1 31  1  1  1  1]
471: [ 1 12  1 37 13 32  1  1  1  1]
472: [ 1  1 15 43 35  1  1  1  1  1]
473: [ 1  1 40 27 27  1  1  1  1  0]
4

array([[27, 31, 38,  1,  1,  1,  1,  0,  0,  0]], dtype=uint8)

In [109]:
testbank = strategy_bank(512)

In [115]:
generation_score([27,31,38,1,1,1,1,0,0,0],testbank)

25952

In [113]:
numwins([27,31,38,1,1,1,1,0,0,0],testbank)

477

Although it loses to my original strategy one-on-one, the point of the problem is to submit a strategy that maximises average score against a range of strategies, which is perhaps something I failed to appreciate initially. This strategy also passes the test of beating all 'obvious' strategies.

I think I will trust the computer in this case, and not modify the strategy. So [27,31,38,1,1,1,1,0,0,0] is my submission